In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install anesthetic
import os
os.chdir("/content/drive/MyDrive/project")

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import torch
import torch.nn as nn
import torch.distributions as dists
import torch.optim as optim
from torch.distributions import MultivariateNormal
from torch.distributions.distribution import Distribution
from np_utils import simple_data, curved_data, planck_des_data
from torch_utils import rotation_test, get_limits, TrainUtil
from tension_net import TensionNet, TensionNet1, TensionNet2, TensionNet3
from tension_quantify import GaussianKDE, BayesFactorKDE, BayesFactor

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
(X1, X1_weights, X0, X0_weights,
 X_prior, params) = planck_des_data(only_lcdm=False)
weights = { "XA": X0_weights, "XB": X1_weights }

In [ ]:
start = time.time()
tension_R = TensionNet1(6, hidden_size=64)
criterion = BayesFactorKDE(device, n_points=1000, logsumexp=True)
optimizer = optim.Adam(tension_R.parameters(), lr=0.001)

train_util_R = TrainUtil(tension_R, optimizer, criterion, device)
losses_R = train_util_R.train(X0, X1, X_prior, weights=weights, n_iter=1000)
print("KDE time taken: ", time.time() - start)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

train_util_R_1.plot_loss(axs[0])
train_util_R_1.plot_marginalised_dists(axs[1])
train_util_R_1.plot_marginalised_dists(axs[2], flat_prior=True)
fig.savefig("images/loss_lcdm_3.png")

In [ ]:
fig, axs = plt.subplots(6, 6, figsize=(15, 15), sharey='row', sharex='col')
fig.suptitle(f"Contour plot of tension coordinate, with Bayes Factor as loss function. R = {round(losses_R_1[-1], 4)}")
train_util_R_1.visualise_coordinates_all(fig, axs, param_names=params,
                                       sync_levels=True, tension_as_param=True)
fig.savefig("images/contour_lcdm_3.png")